In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.conv_block(x)
        p = self.pool(x)
        return x, p

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_block = ConvBlock(in_channels, out_channels)
    
    def forward(self, x, skip):
        x = self.upconv(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv_block(x)
        return x

In [ ]:
class AttentionGate(nn.Module):
    def __init__(self, in_channels, gating_channels, inter_channels):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Conv2d(gating_channels, inter_channels, kernel_size=1)
        self.W_x = nn.Conv2d(in_channels, inter_channels, kernel_size=1)
        self.psi = nn.Conv2d(inter_channels, 1, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, g):
        # g: gating signal (from decoder), x: input (from encoder)
        g = self.W_g(g)
        x = self.W_x(x)
        
        # Attention gate calculation
        additive = F.relu(g + x)
        psi = self.psi(additive)
        attention = self.sigmoid(psi)
        
        return x * attention

In [ ]:
class AttentionUNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(AttentionUNet, self).__init__()
        
        # Encoder (Contracting Path)
        self.enc1 = DownBlock(in_channels, 64)
        self.enc2 = DownBlock(64, 128)
        self.enc3 = DownBlock(128, 256)
        self.enc4 = DownBlock(256, 512)
        
        # Bottleneck
        self.bottleneck = ConvBlock(512, 1024)
        
        # Decoder (Expansive Path)
        self.up4 = UpBlock(1024, 512)
        self.up3 = UpBlock(512, 256)
        self.up2 = UpBlock(256, 128)
        self.up1 = UpBlock(128, 64)
        
        # Final convolution for segmentation output
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Attention Gates
        self.att_gate4 = AttentionGate(512, 1024, 256)
        self.att_gate3 = AttentionGate(256, 512, 128)
        self.att_gate2 = AttentionGate(128, 256, 64)
        self.att_gate1 = AttentionGate(64, 128, 32)

    def forward(self, x):
        # Encoder
        enc1, p1 = self.enc1(x)
        enc2, p2 = self.enc2(p1)
        enc3, p3 = self.enc3(p2)
        enc4, p4 = self.enc4(p3)
        
        # Bottleneck
        bottleneck = self.bottleneck(p4)
        
        # Decoder with attention gates
        up4 = self.up4(bottleneck, self.att_gate4(enc4, bottleneck))
        up3 = self.up3(up4, self.att_gate3(enc3, up4))
        up2 = self.up2(up3, self.att_gate2(enc2, up3))
        up1 = self.up1(up2, self.att_gate1(enc1, up2))
        
        # Final segmentation output
        segmentation_output = self.final_conv(up1)

        return segmentation_output